In [2]:
import pyspark as pys
import pyspark.sql.functions as sf
import pyspark.sql as s
import datetime as dt
from pyspark import SparkContext
import pyspark.sql.types as st
import logging

AWS_ACCESS_KEY=''
AWS_SECRET_KEY=''

MAX_MEMORY = "5g"

spark = pys.sql.SparkSession.builder \
    .master('local[*]') \
    .config("spark.jars.packages", "com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.3")\
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", AWS_ACCESS_KEY) 
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", AWS_SECRET_KEY)

spark.conf.set("spark.sql.broadcastTimeout", 1200)

LOCAL_PATH = "~/home/where_you_wish"

In [3]:
def config_log():
    from sys import stdout
    stdout_handler = logging.StreamHandler(stdout)
    handlers = [stdout_handler]
    logging.basicConfig(
        level=logging.INFO,
        format='[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
        handlers=handlers
    )
    return logging.getLogger('JOB')

logger = config_log()

## Transient to Raw

In [ ]:
def args_datasets_read_types(sys_args):
    """ param sys_args: Receive all atributos from Glue UI param.
        All datasets with "--n_" prefixes are being treating
        with datasets and read types.
        e.g:
            --n_consumer / csv.
            --n_order / json
            --n_....
        return dict with datasets and read types.
    """
    datasets_read_types = []
    for liu in enumerate(sys_args):
        if '--n_' in liu[1]:
            dataset = liu[1][4:]
            read_type = sys_args[liu[0]+1]
            datasets_read_types.append([dataset, read_type])
    return datasets_read_types

In [ ]:
def datetime_result(date_type):
    """ Recover brazilian time.
    """
    if date_type == 'today':
        date_now = (
            dt.datetime.utcnow() - dt.timedelta(hours=3)).strftime('%Y-%m-%d')
        return str(date_now)
    if date_type == 'today_timestamp':
        date_now = (
            dt.datetime.utcnow() - dt.timedelta(hours=3)) \
            .strftime('%Y-%m-%d %H:%M:%S')
        return str(date_now)
    else:
        date_yesternoon = (
            dt.datetime.utcnow() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
        return str(date_yesternoon)


In [ ]:
def path_read_write(zone, dataset_name, read_type):
    """
        refurn absolute bucket path.
    """
    BASE_PATH_SOURCE = 's3a://ifood-data-architect-test-source'

    if zone == 'source':
        PATH = BASE_PATH_SOURCE + '/' + dataset_name + '.' + read_type + '.gz'
    else:
        PATH = LOCAL_PATH + zone + '/' + dataset_name

    return PATH

In [ ]:
def read_dataset(zone, dataset_name, read_type):
    """
    e.g:
        Read json and csv from source repo, creating
        partition column setting now date.
        return spark dataframe
    """
    PARTITION_DATE = datetime_result('today')
    PATH = path_read_write(zone, dataset_name, read_type)

    READ = spark.read \
        .load(PATH, format=read_type, header=True, infer_schema=True) \
        .withColumn('partition_column', sf.lit(PARTITION_DATE))

    return READ

In [ ]:
def write_dataset(zone, dataset_name, df):
    """ param zone (transient/raw/refined/trusted)
        param dataset_name
        param df (spark dataframe)
        e.g:
            Write some data in .parquet extension,
            overwriting existing data inside the partition.
        return spark dataframe
    """
    PATH = path_read_write(zone, dataset_name, '.parquet')
    df.write \
      .mode('overwrite') \
      .partitionBy('partition_column') \
      .parquet(PATH)


In [4]:
parameters = ['--n_order','json','--n_consumer','csv','--n_status', 'json', '--n_restaurant', 'csv']

In [ ]:
for dataset_name, type_read in args_datasets_read_types(parameters):
    """ Iterate from all datasets and read_types configurable.
    """

    logger.info('Load Transient:' + datetime_result('today_timestamp') + ' - Dataset: ' + dataset_name + ' - type_read: ' + type_read)
    df = read_dataset('source', dataset_name, type_read)

    logger.info('Write Raw: ' + datetime_result('today_timestamp') + ' - Dataset: ' + dataset_name + ' - type_read: ' + type_read)
    write_dataset('raw', dataset_name, df)

## Raw to Trusted

In [ ]:
def datetime_result(date_type):
    """ Recover brazilian time.
    """ 
    if date_type == 'today':
        date_now = (dt.datetime.utcnow() - dt.timedelta(hours=3)).strftime('%Y-%m-%d')
        return str(date_now)
    if date_type == 'today_timestamp':
        date_now = (dt.datetime.utcnow() - dt.timedelta(hours=3)).strftime('%Y-%m-%d %H:%M:%S')
        return str(date_now)
    else:
        date_yesternoon = (dt.datetime.utcnow() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
        return str(date_yesternoon)

In [ ]:
def hash_column(df, columns):
    """
    Used to hash columns
    :param df: :class:`pyspark.sql.DataFrame` instance
    :param columns: list, columns to hash
    :return df: :class:`pyspark.sql.DataFrame` instance
    """
    salt = 'QglW6kADoUt8-FXIHQQsj3tK-Vpz6QaZ2DoCCQKEARM='
    for column in columns:
        df = df.withColumn(column, sf.trim(sf.col(column)))
        df = df.withColumn(column, sf.concat(column, sf.lit(salt)))
        df = df.withColumn(column, sf.sha2(column, 256))
    return df

In [ ]:
@sf.udf
def hash_tel(column):
    """
    Def for anonymized phone
    """
    return int(column) * 5 / 2

In [ ]:
def path_read_write(zone, dataset_name):
    """
    Def mount absolute bucket path.
    """
    BASE_PATH_SOURCE = LOCAL_PATH

    PATH = BASE_PATH_SOURCE + '/' + zone + '/' + dataset_name

    return PATH

In [ ]:
def dedup_dataframe(pk_col, max_col, dataframe):
    """
    Def for deduplication remaining the most recent record
    """
    LAST_VALUE_ORDER = s.Window.partitionBy(
        pk_col) \
        .orderBy(sf.col(max_col).desc())

    DF_DEDUP = dataframe.dropDuplicates() \
        .withColumn("distinct", sf.row_number().over(LAST_VALUE_ORDER)) \
        .filter("distinct = 1") \
        .drop("distinct")

    return DF_DEDUP

In [ ]:
def checking_quality(actual, previous):
    """
    :param actual: number of records comming from last trusted zone.
    :param previous: number of records comming from the wildcard previues trusted zone.
    :return: TRUE, if the dataset is OK to be inserted / FALSE, if any problem
    related to quality was found.
    """
    # Checking the percentage of drop in the counting
    maximum_percentage_of_drop_in_count = 80

    logger.info(f"Quality Message: Actual {actual} - Previous {previous}")
    
    if actual < previous:
        if ((((actual / previous) - 1) * 100) * -1) >= maximum_percentage_of_drop_in_count:  # noqa
            return False

    return True

In [ ]:
def write_trusted(dataframe, repartition, partition, dataset):
    
    PATH = path_read_write("trusted", dataset)
    try:
        # Todo, find last dataset wildcard using boto3 for figureout that.
        DF_LATEST = spark.read \
            .parquet(PATH + '/partition=' + datetime_result("yesterday"))
        latest_v2_count = DF_LATEST.count()
    
    except Exception:
        latest_v2_count = 1
    
    logger.info("Getting the count from the current versions of the trusted")
    latest_count = dataframe.count()
    quality_check = checking_quality(latest_count, latest_v2_count)
    
    if quality_check:
        
        dataframe.repartition(repartition).write \
        .parquet(PATH + '/partition=' + datetime_result("today"),
             mode="overwrite",
             partitionBy=partition)
    else:
        logger.info(f"Quality Urgent: Dataset: {dataset} are 80% lower. Check it out!!")

In [ ]:
# Setting for read all datasets in raw zone
RAW_ORDER = spark.read \
    .load(path_read_write("raw", "order"), format='parquet') \
    .drop("partition_column", "customer_name") \

RAW_STATUS = spark.read \
    .load(path_read_write("raw", "status"), format='parquet') \
    .drop("partition_column") \
    .withColumnRenamed("created_at", "status_created_at") \
    .withColumnRenamed("value", "status_value") \

RAW_CONSUMER = spark.read \
    .load(path_read_write("raw", "consumer"), format='parquet') \
    .drop("partition_column", "customer_name") \
    .withColumnRenamed("created_at", "consumer_created_at") \
    .withColumn("customer_phone_number", hash_tel('customer_phone_number'))

RAW_RESTAURANT = spark.read \
    .load(path_read_write("raw", "restaurant"), format='parquet') \
    .drop("partition_column") \
    .withColumnRenamed("created_at", "restaurant_created_at")

In [ ]:
def trusted_order():

    # Join Order + Consumer + Restaurant
    DF_PRIMARY = RAW_ORDER \
        .join(RAW_CONSUMER, 'customer_id') \
        .join(RAW_RESTAURANT, RAW_RESTAURANT.id == RAW_ORDER.merchant_id)

    # Read Order dataset
    DF_TRUSTED_ORDER = dedup_dataframe(
        "customer_id",
        "order_created_at",
        DF_PRIMARY)

    # Read Status dataset
    DF_TRUSTED_STATUS = dedup_dataframe(
        "order_id",
        "status_created_at",
        RAW_STATUS) \
        .select("order_id", "status_created_at", "status_value")

    # Joining
    DF_TRUSTED_FINAL = DF_TRUSTED_ORDER \
        .join(DF_TRUSTED_STATUS, "order_id") \
        .withColumn("partition",
                    sf.lit(datetime_result("today")))

    DF_TRUSTED_ANONYMIZED = hash_column(DF_TRUSTED_FINAL, ['cpf'])

    # Writing DF in trusted zone.
    write_trusted(DF_TRUSTED_ANONYMIZED, 30, "partition", "order")

In [ ]:

    PATH_ORDER = path_read_write("trusted", "order")

    DF_TRUSTED = spark.read \
        .parquet(PATH_ORDER)

    # Recovering dynamic schema
    JSON_SCHEMA = spark.read \
        .json(DF_TRUSTED.rdd.map(lambda row: row.items)).schema

    # Inferring schema in string column, to become array para usar explode code
    JSON_SCHEMA = st.ArrayType(JSON_SCHEMA)
    DF_FLATTEN = DF_TRUSTED \
        .withColumn("items", sf.from_json("items", JSON_SCHEMA))

    # Exploding and struturing Items for construct Order Items
    DF_EXPLODE_ITEMS = DF_FLATTEN.select(
        "order_id",
        sf.explode("items").alias("items"))

    # Structuring data
    DF_EXPLODE_ITEMS_DETAILS = DF_EXPLODE_ITEMS \
        .select("order_id",
                sf.col("items.name").alias("name"),
                sf.col("items.addition")
                .getItem('currency').alias("addition_currency"),
                sf.col("items.addition").getItem('value').alias("addition"),
                sf.col("items.discount")
                .getItem('currency').alias("discount_currency"),
                sf.col("items.discount").getItem('value').alias("discount"),
                sf.col("items.quantity").alias("quantity"),
                sf.col("items.unitPrice")
                .getItem('currency').alias("unit_price_currency"),
                sf.col("items.unitPrice").getItem('value').alias("unit_price"),
                sf.col("items.externalId").alias("external_id"),
                sf.col("items.totalValue")
                .getItem('currency').alias("total_value_currency"),
                sf.col("items.totalValue")
                .getItem('value').alias("total_value"),
                sf.col("items.customerNote").alias("customer_note"),
                sf.col("items.integrationId").alias("integration_id"),
                sf.col("items.totalAddition")
                .getItem('currency').alias("tota_addition_currency"),
                sf.col("items.totalAddition")
                .getItem('value').alias("tota_addition"),
                sf.col("items.totalDiscount")
                .getItem('currency').alias("total_discount_currency"),
                sf.col("items.totalDiscount")
                .getItem('value').alias("total_discount"),
                sf.explode("items.garnishItems.externalId")
                .alias("garnish_external_id"),
                sf.col("items.garnishItems").alias("garnish_items")
                ) \
        .dropDuplicates()

    # Exploding and struturing Garnish for additing Order Items
    DF_EXPLODE_GARNISH = DF_EXPLODE_ITEMS_DETAILS.select(
        "order_id",
        "garnish_external_id",
        sf.explode("garnish_items").alias("garnish_items"))

    # Structuring data
    DF_EXPLODE_GARNISH_DETAIL = DF_EXPLODE_GARNISH \
        .select("order_id",
                sf.col("garnish_items.name").alias("garnish_name"),
                sf.col("garnish_items.addition")
                .getItem('value').alias("garnish_addition"),
                sf.col("garnish_items.addition")
                .getItem('currency').alias("garnish_addition_currency"),
                sf.col("garnish_items.discount")
                .getItem('value').alias("garnish_discount"),
                sf.col("garnish_items.discount")
                .getItem('currency').alias("garnish_discount_currency"),
                sf.col("garnish_items.quantity").alias("garnish_quantity"),
                sf.col("garnish_items.sequence").alias("garnish_sequence"),
                sf.col("garnish_items.unitPrice")
                .getItem('value').alias("garnish_unit_price"),
                sf.col("garnish_items.unitPrice")
                .getItem('currency').alias("garnish_unit_price_currency"),
                sf.col("garnish_items.categoryId")
                .alias("garnish_category_id"),
                sf.col("garnish_items.categoryName")
                .alias("garnish_category_name"),
                sf.col("garnish_items.externalId")
                .alias("garnish_external_id"),
                sf.col("garnish_items.totalValue")
                .getItem('value').alias("garnish_total_value"),
                sf.col("garnish_items.totalValue")
                .getItem('currency').alias("garnish_total_value_currency"),
                sf.col("garnish_items.integrationId")
                .alias("garnish_integration_id"),
                ) \
        .dropDuplicates()

    # Union itens with garnish dataframe,
    # when 1 or more item can have 1 or more garnish.
    DF_UNION_ITENS_GARNISH = DF_EXPLODE_ITEMS_DETAILS \
        .join(DF_EXPLODE_GARNISH_DETAIL, ["order_id", "garnish_external_id"]) \
        .drop("garnish_items") \
        .withColumn("partition",
            sf.lit(datetime_result("today")))

    # Writing DF in trusted zone.
    write_trusted(DF_UNION_ITENS_GARNISH, 10, "partition", "order_items")

In [ ]:
def trusted_order_status():

    PATH_ORDER = path_read_write("trusted", "order")
    DF_ORDER_STATUSES = spark.read \
        .parquet(PATH_ORDER) \

    # Creating temp view in memory for use in query below
    DF_ORDER_STATUSES.createOrReplaceTempView("order")
    RAW_STATUS.createOrReplaceTempView("status")

    DF_STATUSES_ORDER = spark.sql("""
    select  ord.order_id,
            st1.status_value status_1,
            st1.status_created_at status_created_at_1,
            st2.status_value status_2,
            st2.status_created_at status_created_at_2,
            st3.status_value status_3,
            st3.status_created_at status_created_at_3,
            st4.status_value status_4,
            st4.status_created_at status_created_at_4
    from order ord
    left join (
        select status_created_at, status_value, order_id
          from status
         where status_value = 'REGISTERED') st1 on st1.order_id = ord.order_id

    left join (
        select status_created_at, status_value, order_id
          from status
         where status_value = 'PLACED') st2 on st2.order_id = ord.order_id

    left join (
        select status_created_at, status_value, order_id
          from status
         where status_value = 'CONCLUDED') st3 on st3.order_id = ord.order_id

    left join (
        select status_created_at, status_value, order_id
          from status
         where status_value = 'CANCELLED') st4 on st4.order_id = ord.order_id
    """).dropDuplicates() \
    .withColumn("partition", sf.lit(datetime_result("today")))

    # Writing DF in trusted zone.
    write_trusted(DF_STATUSES_ORDER, 1, "partition", "order_statuses")

In [ ]:
trusted_order()
trusted_order_items()
trusted_order_status()